In [ ]:
from models.ABrain import Airfoil_Brain
import numpy as np
import matplotlib.pyplot as plt
import torch

# Initialize
current_ada_iter=65
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
airfoil_brain=Airfoil_Brain(model_path_header='./model_save/pp_model_save/ada{:d}'.format(current_ada_iter), device=device) # load prior model
airfoil_brain.std_var='uniform'


In [ ]:
fitted_latent=np.load('fitted_latent_naca23012.npy')
fitted_latent2=np.load('fitted_latent_clarky.npy')
fitted_latent3=np.load('fitted_latent_sm701.npy')

fitted_latents=np.concatenate([fitted_latent, fitted_latent2, fitted_latent3])

In [ ]:
Ma_grid=np.array([0.3, 0.5, 0.75])
AoA_grid=np.linspace(-20,25,91)
latent=np.zeros((AoA_grid.shape[0], 13))

marker_seq=['o', 's', '<', '^','>']
color_seq=['darkblue', 'crimson', 'forestgreen', 'orangered', 'teal']
plt.rcParams['font.family']='times new roman'
plt.rcParams['font.size']=16

plt.figure(figsize=(12,6), dpi=450)
plt.subplots_adjust(wspace=0.3)
for k in range(fitted_latents.shape[0]):
    plt.subplot(1,3,k+1)
    for j in range(Ma_grid.shape[0]):
        for i in range(AoA_grid.shape[0]):
            latent[i,:10]=fitted_latents[k]
            latent[i,10]=Ma_grid[j]
            latent[i,11]=AoA_grid[i]
            latent[i,12]=5e6
        airfoil, mean, std1, std2 = airfoil_brain.evaluate(latent, norm=True)
        mean[:,1]=10**(mean[:,1])
        
        plt.plot(mean[:,1], mean[:,0],'-', marker=marker_seq[j], markerfacecolor='none', color='k')
        plt.xlim(0, 0.1)
        plt.ylim(-1.5, 2.0)
        plt.grid(True,alpha=0.8)
    plt.plot(0.045+airfoil[0,:,0]*0.05, -1.3+airfoil[0,:,1],'k')

plt.show()
